# BLACK_SCHOLES

## Overview
The `BLACK_SCHOLES` function calculates the theoretical price of a European call or put option using the Black-Scholes formula. This enables users to quickly and accurately price options for financial analysis, risk management, or portfolio valuation.

The Black-Scholes model is a mathematical model for pricing European-style options and is widely used in financial engineering. The model assumes that the price of the underlying asset follows a geometric Brownian motion with constant drift and volatility. The key calculations performed by the `black_scholes` function are:

- **Call Option Price:** The price to buy the right, but not the obligation, to purchase an asset at a specified strike price (K) on the option's expiration date. The Black-Scholes formula for a European call option is:

```math
C = S N(d_1) - K e^{-rT} N(d_2)
```

- **Put Option Price:** The price to buy the right, but not the obligation, to sell an asset at a specified strike price (K) on the option's expiration date. The Black-Scholes formula for a European put option is:

```math
P = K e^{-rT} N(-d_2) - S N(-d_1)
```

where:
- $S$: Current price of the underlying asset (e.g., stock price)
- $K$: Strike price of the option
- $T$: Time to expiration in years
- $r$: Annual risk-free interest rate (as a decimal)
- $\sigma$: Volatility of the underlying asset (annualized standard deviation, as a decimal)
- $N(\cdot)$: Cumulative distribution function of the standard normal distribution
- $d_1 = \frac{\ln(S/K) + (r + 0.5 \sigma^2) T}{\sigma \sqrt{T}}$
- $d_2 = d_1 - \sigma \sqrt{T}$

**Assumptions:**
- The option is European (can only be exercised at expiration)
- No dividends are paid out during the life of the option
- Markets are efficient (market movements cannot be predicted)
- No commissions or transaction costs
- The risk-free rate and volatility of the underlying asset are constant
- The returns of the underlying asset are normally distributed

## Usage
To use the function in Excel:

```excel
=BLACK_SCHOLES(S, K, T, r, sigma, [option_type])
```
- `S` (float, required): Current stock price. Example: `100`
- `K` (float, required): Strike price of the option. Example: `100`
- `T` (float, required): Time to expiration in years. Example: `1`
- `r` (float, required): Risk-free interest rate. Example: `0.05`
- `sigma` (float, required): Volatility of the stock. Example: `0.2`
- `option_type` (string, optional): Type of option ("call" or "put"). Example: `"call"`

The function returns the Black-Scholes option price as a float, or an error message as a string if the input is invalid.

## Examples
- `=BLACK_SCHOLES(100, 105, 0.5, 0.03, 0.2, "call")` → `4.18`
- `=BLACK_SCHOLES(50, 45, 1, 0.02, 0.25, "put")` → `2.31`


In [ ]:
import math
from scipy.stats import norm

def black_scholes(S, K, T, r, sigma, option_type="call"):
    """
    Calculate the Black-Scholes price for a European call or put option.

    Args:
        S (float): Current price of the underlying asset.
        K (float): Strike price.
        T (float): Time to expiration in years.
        r (float): Annual risk-free interest rate (decimal).
        sigma (float): Volatility of the underlying asset (decimal).
        option_type (str, optional): 'call' or 'put'. Defaults to 'call'.

    Returns:
        float: Theoretical price of the option, or
        str: Error message if calculation fails.

    This example function is provided as-is without any representation or warranty of correctness. DO NOT USE for production applications without your own testing and validation.
    """
    if not (isinstance(S, float) and S > 0):
        return "Error: S must be a positive float"
    if not (isinstance(K, float) and K > 0):
        return "Error: K must be a positive float"
    if not (isinstance(T, float) and T > 0):
        return "Error: T must be a positive float"
    if not (isinstance(r, float) and r >= 0):
        return "Error: r must be a non-negative float"
    if not (isinstance(sigma, float) and sigma > 0):
        return "Error: sigma must be a positive float"
    if option_type not in ("call", "put"):
        return "Error: option_type must be 'call' or 'put'"

    d1 = (math.log(S / K) + (r + 0.5 * sigma ** 2) * T) / (sigma * math.sqrt(T))
    d2 = d1 - sigma * math.sqrt(T)
    if option_type == "call":
        price = S * norm.cdf(d1) - K * math.exp(-r * T) * norm.cdf(d2)
    else:
        price = K * math.exp(-r * T) * norm.cdf(-d2) - S * norm.cdf(-d1)
    return float(price)

In [ ]:
%pip install -q ipytest
import ipytest
ipytest.autoconfig()

demo_cases = [
    [100, 105, 0.5, 0.03, 0.2, "call"],
    [50, 45, 1, 0.02, 0.25, "put"],
    [100, 100, 1, 0.05, 0.2, "call"],
    [100, 100, 1, 0.05, 0.2, "put"],
]

def is_valid_type(val):
    if isinstance(val, (float, bool, str)):
        return True
    if isinstance(val, list):
        return all(isinstance(row, list) and all(isinstance(x, (float, bool, str)) for x in row) for row in val)
    return False

import pytest
@pytest.mark.parametrize("S, K, T, r, sigma, option_type", demo_cases)
def test_demo_cases(S, K, T, r, sigma, option_type):
    result = black_scholes(S, K, T, r, sigma, option_type)
    assert is_valid_type(result), f"Output type is not valid. Got: {type(result)} Value: {result}"

def test_invalid_negative_stock():
    result = black_scholes(-100, 100, 1, 0.05, 0.2, "call")
    assert isinstance(result, str) and result.startswith("Error:")

def test_invalid_option_type():
    result = black_scholes(100, 100, 1, 0.05, 0.2, "invalid")
    assert isinstance(result, str) and result.startswith("Error:")

ipytest.run('-s')

In [ ]:
# Gradio demo for black_scholes function
import gradio as gr

demo = gr.Interface(
    fn=black_scholes,
    inputs=[
        gr.Number(label="S (Stock Price)", value=demo_cases[0][0]),
        gr.Number(label="K (Strike Price)", value=demo_cases[0][1]),
        gr.Number(label="T (Time to Expiration, years)", value=demo_cases[0][2]),
        gr.Number(label="r (Risk-Free Rate)", value=demo_cases[0][3]),
        gr.Number(label="sigma (Volatility)", value=demo_cases[0][4]),
        gr.Dropdown(["call", "put"], label="Option Type", value=demo_cases[0][5]),
    ],
    outputs=gr.Number(label="Option Price"),
    examples=demo_cases,
    description="Calculate the theoretical price of a European call or put option using the Black-Scholes formula.",
    flagging_mode="never",
    fill_width=True,
)

demo.launch()